# Initialization

In [ ]:
import bgk

In [ ]:
path = f"/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/max/B02.00-n256/"
# path = f"/mnt/lustre/IAM851/jm1667/psc-runs/tests/test14a/"

run_manager = bgk.RunManager(path)
params_record = run_manager.params_record
run_diagnostics = run_manager.run_diagnostics

size = run_diagnostics.domain_size
struct_radius = run_diagnostics.hole_radius

wholeSlice = bgk.DataSlice(slice(None, None), "")
centerSlice = bgk.DataSlice(slice(-struct_radius, struct_radius), "Central ")

run_diagnostics.print_params()
run_diagnostics.check_params()

In [ ]:
# fiddle with this until as many steps as possible are used (usually, they can all be used)
nframes = 1001

videoMaker = bgk.VideoMaker(nframes, run_manager)

videoMaker.frame_manager.print_coverage()

# Animation
from https://stackoverflow.com/questions/18743673/show-consecutive-images-arrays-with-imshow-as-repeating-animation-in-python
and http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/

In [ ]:
from IPython.display import HTML
import bgk.run_params as rp

In [ ]:
# select parameter
param = rp.ne
print(f"quantity: {param.title}")

In [ ]:
# load data
videoMaker.loadData(param)

In [ ]:
# slice data
sliceId = 0
whichSlice = [wholeSlice, centerSlice][sliceId]
videoMaker.setSlice(whichSlice)

print(f"view: {whichSlice.viewAdjective}= {whichSlice.slice}")

In [ ]:
# view t=0
%matplotlib widget
fig, ax, im = videoMaker.viewFrame(0)

In [ ]:
# make movie
%matplotlib widget
anim = videoMaker.viewMovie(fig, ax, im)
HTML(anim.to_html5_video())

## Stability Plot

In [ ]:
%matplotlib widget
fig, ax = videoMaker.viewStability()

In [ ]:
%matplotlib widget
fig, ax = videoMaker.viewMeansAtOrigin()

# Radial Dependence

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

maxR = whichSlice.slice.stop
rStep = size / 100

def getMeanAndStd(data: xr.DataArray, r: float) -> tuple[float, float]:
    rslice = data.where((r <= videoMaker.grid_rho) & (videoMaker.grid_rho < r + rStep))
    return rslice.mean().item(), rslice.std().item()

quantiles = np.linspace(0, 1, 11)
def getPercentiles(data, r):
    rslice = data.where((r <= videoMaker.grid_rho) & (videoMaker.grid_rho < r + rStep))
    return tuple(np.nanquantile(rslice.values, p) for p in quantiles)

rs = np.arange(0, maxR, rStep)
def getMeansAndStds(data):
    return tuple(zip(*[getMeanAndStd(data, r) for r in rs]))
def getPercentileses(data):
    return tuple(zip(*[getPercentiles(data, r) for r in rs]))

### Time-Averaged

In [ ]:
allMeans = np.array([getMeansAndStds(videoMaker.datas[idx])[0] for idx in range(nframes)])

In [ ]:
time_cutoff_idx = len(videoMaker.datas) - 1
titleText = "over Run"
# time_cutoff_idx = videoMaker.getIdxPeriod()
# titleText = "over First Oscillation"

In [ ]:
# A handful of samples
%matplotlib widget
plt.close("all")

nsamples = 13
indices = sorted(list({round(i) for i in np.linspace(0, time_cutoff_idx, nsamples)}))
# indices = range(nsamples)

n_label_indices = 5
label_indices = [indices[round(i * (len(indices) - 1) / (n_label_indices - 1))] for i in range(n_label_indices)]

fig, ax = plt.subplots()

def plot_lines(indices, cmap, label_indices):
    for i in indices:
        label = f"$t={videoMaker.times[i]:.2f}$" if i in label_indices else "_nolegend_"
        ax.plot(rs, allMeans[i], color=cmap(i / max(indices)), label=label)

def get_cmap(name: str, min: float=0.0, max: float=1.0, reverse: bool=False):
    return lambda x: matplotlib.colormaps[name](min + (1-x if reverse else x) * (max - min))

cmap = get_cmap("rainbow")

plot_lines(indices, cmap, label_indices)

ax.set_xlabel("$\\rho$")
ax.set_ylabel(param.title)
ax.set_title(f"Extremal Profiles of {param.title} for $B_0={params_record.B0}$ over Run")
ax.legend()
fig.tight_layout()

In [ ]:
fig.savefig("figs-test/whatever.png", bbox_inches="tight", pad_inches=0.01, dpi=300)

In [ ]:
# Min, mean, and max
%matplotlib widget

plt.plot(rs, allMeans.max(axis=0), color="lightcoral", label="max mean")
plt.plot(rs, allMeans[0], color="blue", linestyle=":", label="$t=0$")
plt.plot(rs, allMeans.mean(axis=0), color="red", label="mean mean")
plt.plot(rs, allMeans.min(axis=0), color="lightcoral", label="min mean")

plt.xlabel("$\\rho$")
plt.ylabel(param.title)
plt.title(f"Temporal & Radial Dependence of {param.title}, $B_0={params_record.B0}$")
plt.legend()
plt.tight_layout()
plt.show()

### Mean & Error

In [ ]:
dataIdx = -1
means, stds = getMeansAndStds(videoMaker.datas[dataIdx])
means0, stds0 = getMeansAndStds(videoMaker.datas[0])

In [ ]:
%matplotlib widget

plt.xlabel("Distance from Axis")
plt.ylabel(param.title)
plt.title("Mean " + param.title + " vs Radius (t={:.2f})".format(videoMaker.times[dataIdx]))

plt.errorbar(rs, means0, yerr=stds0, errorevery=(1,2), color="blue", ecolor="lightskyblue", elinewidth=1, capsize=1.5, label=f"t={0:.2f}")
plt.errorbar(rs, means, yerr=stds, errorevery=(0, 2), color="red", ecolor="lightcoral", elinewidth=1, capsize=1.5, label=f"t={videoMaker.times[dataIdx]:.2f}")
plt.legend()
plt.show()

### Percentiles

In [ ]:
dataIdx = -1
percentiles = getPercentileses(videoMaker.datas[dataIdx])

In [ ]:
%matplotlib widget

plt.xlabel("$\\rho$")
plt.ylabel(param.title)
plt.title(f"{len(quantiles)} Percentiles of " + param.title + " vs $\\rho$ (t={:.2f})".format(videoMaker.times[dataIdx]))

cmap = matplotlib.cm.get_cmap('rainbow')

labels = ["{:.0f}%".format(q*100) for q in quantiles]
colors = [cmap(2*abs(.5-q)) for q in quantiles]

for perc, color, label in reversed(list(zip(percentiles, colors, labels))):
    plt.plot(rs, perc, color=color, label=label)

# plt.legend()
plt.show()